In [27]:
import sys
sys.path.append("..")
import functools
import pickle

import numpy as np
import torch
import pyequion2

import tmcdiff

In [28]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
TK = 298.15
bulk_velocity = 1e0
pipe_diameter = 1e-2
species = ["C", "Ca", "Na", "Cl", "Mg"]
phases = ["Calcite", "Dolomite"]
c_nahco3 = 0.018
c_cacl2 = 0.006
c_nacl = 0.003
c_mgcl2 = 0.012
molal_balance = {'C':c_nahco3,
                 'Ca':c_cacl2,
                 'Na':c_nahco3 + c_nacl,
                 'Cl':2*c_cacl2 + c_nacl + 2*c_mgcl2,
                 'Mg': c_mgcl2}
ngrid = 20
ypmax = 5

In [30]:
pyequion2.constants.GAS_CONSTANT

8.31446261815324

In [31]:
def get_reaction_constant(A, E, TK):
    logkr = np.log(A) - E/8.31446261815324*(1/TK - 1/298.15)
    return np.exp(logkr)

kr = get_reaction_constant(*(52153.84138874875, 86881.05), 298.15)*1e-6
kr, kr/2

(0.05215384138874875, 0.026076920694374374)

In [32]:
solver = tmcdiff.TransportSolver(species)
solver.kreaction = np.array([kr, kr/2])
solver.set_flow_conditions(TK, bulk_velocity, pipe_diameter)
solver.set_initial_conditions(molal_balance, phases)
solver.build_transport(ngrid, ypmax)
solver.set_initial_guess()

In [33]:
eqsys_front = pyequion2.EquilibriumSystem(species, from_elements=True,
                                           activity_model="DEBYE")
sol, _ = eqsys_front.solve_equilibrium_mixed_balance(TK, molal_balance=molal_balance)
sol.saturation_indexes

{'Aragonite': 1.2148915253441697,
 'Calcite': 1.358661372736405,
 'Dolomite': 3.155007319921962,
 'Halite': -4.887159336866009,
 'Vaterite': 0.7922298785221056}

In [34]:
solver.solve(100.0, print_frequency=1)

0.002774054746908511%
0.030545479128457412%
0.23154041608741416%
0.23465154726727264%
0.23776267844713111%
0.2688739902457158%
0.5799871082315624%
0.6422097318287318%
0.7044323554259012%
1.3266585913975946%
1.328726390518306%
1.3307941896390174%
1.351472180846132%
1.55825209291728%
1.7650320049884285%
1.9718119170595767%
2.178591829130725%
2.2515899769594405%
2.324588124788156%
3.0545696030753122%
3.0568181193986366%
3.0590666357219605%
3.081551798955206%
3.3064034312876593%
3.35137375775415%
3.3963440842206403%
3.8460473488855467%
3.9584731650517733%
4.070898981218%
4.377196824778437%
4.419868111843923%
4.462539398909408%
4.889252269564263%
4.974594843695234%
5.059937417826205%
5.145279991957176%
5.230622566088147%
5.391911871840568%
5.407986665791371%
5.424061459742172%
5.584809399250193%
5.616958987151796%
5.6491085750534005%
5.9706044540694405%
5.973883830902676%
5.977163207735912%
6.009956976068272%
6.337894659391876%
6.354291543558056%
6.370688427724236%
6.534657269386038%
6.5530

78.25526334661751%
78.35458631690075%
78.45390928718399%
79.4471389900164%
80.09211951246037%
80.42291549355636%
80.75371147465235%
80.87001573023339%
80.98631998581443%
81.10262424139547%
82.26566679720585%
82.49827530836792%
82.73088381952999%
82.96349233069206%
85.28957744231283%
85.29539265509189%
85.30120786787094%
85.35935999566146%
85.94088127356666%
85.95064021735466%
85.96039916114269%
86.0579885990228%
87.03388297782402%
87.04364192161204%
87.05340086540005%
87.15099030328015%
88.12688468208138%
88.37085827678169%
88.614831871482%
91.05456781848503%
91.6645018052358%
92.27443579198655%
93.47919477045258%
93.5032899500219%
93.52738512959121%
93.76833692528442%
96.1778548822165%
96.18013817501179%
96.1824214678071%
96.20525439576028%
96.43358367529201%
98.71687647060939%
98.71918070481978%
98.72148493903018%
98.74452728113414%
98.97495070217383%


In [35]:
solver.save("data/exp3_12")